In [ ]:
#Import the libraries

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report

import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from sklearn.naive_bayes import MultinomialNB


In [ ]:
#Mount the Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Read the 'Career_Data.csv' into df

df = pd.read_csv('/content/drive/MyDrive/capstone/Career_Data.csv')

In [ ]:
df

,text
0,Entry-Level|Data Engineer|WORK EXPERIENCE \nDa...
1,Entry-Level|Data Engineer|WORK EXPERIENCE \n5/...
2,Entry-Level|Data Engineer|WORK EXPERIENCE \nDa...
3,Entry-Level|Data Engineer|WORK EXPERIENCE \nDa...
4,Entry-Level|Data Engineer|WORK EXPERIENCE \nDa...
...,...
150,Senior|DevOps Engineer|WORK EXPERIENCE \nDevOp...
151,Senior|Machine Learning Engineer|WORK EXPERIEN...
152,Entry-Level|Data Engineer|WORK EXPERIENCE \nDa...
153,Entry-Level|Machine Learning Engineer|WORK EXP...


In [ ]:
# Function to extract level, job title, and work experience from each row
def extract_info(row):
    parts = row.split('|')
    level = parts[0]
    job_title = parts[1]
    work_experience = parts[2].split('WORK EXPERIENCE')[1].strip()
    return level, job_title, work_experience

# Apply the extraction function to each row in the DataFrame
df[['Level', 'Job_Title', 'Work_Experience']] = df['text'].apply(extract_info).apply(pd.Series)


In [ ]:
#Drop the original 'text' column
df.drop(columns=['text'], inplace=True)

print(df)

           Level                  Job_Title  \
0    Entry-Level              Data Engineer   
1    Entry-Level              Data Engineer   
2    Entry-Level              Data Engineer   
3    Entry-Level              Data Engineer   
4    Entry-Level              Data Engineer   
..           ...                        ...   
150       Senior            DevOps Engineer   
151       Senior  Machine Learning Engineer   
152  Entry-Level              Data Engineer   
153  Entry-Level  Machine Learning Engineer   
154  Entry-Level            DevOps Engineer   

                                       Work_Experience  
0    Data Engineering Intern \nJuly 2022 – July 202...  
1    5/2022 to 5/2023 1 year\nData Warehouse Engine...  
2    Data Engineer\nApr 2022 Present , 1 yr 4 mos\n...  
3    Data Engineer\nJul 2018 Jun 2019 , 1 yr\nSan F...  
4    Data Engineer\nOct 2017 Jun 2019 , 1 yrs 9 mos...  
..                                                 ...  
150  DevOps Engineer\nJan 2016 to Pr

In [ ]:
df.head()

,Level,Job_Title,Work_Experience
0,Entry-Level,Data Engineer,Data Engineering Intern \nJuly 2022 – July 202...
1,Entry-Level,Data Engineer,5/2022 to 5/2023 1 year\nData Warehouse Engine...
2,Entry-Level,Data Engineer,"Data Engineer\nApr 2022 Present , 1 yr 4 mos\n..."
3,Entry-Level,Data Engineer,"Data Engineer\nJul 2018 Jun 2019 , 1 yr\nSan F..."
4,Entry-Level,Data Engineer,"Data Engineer\nOct 2017 Jun 2019 , 1 yrs 9 mos..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155 entries, 0 to 154
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Level            155 non-null    object
 1   Job_Title        155 non-null    object
 2   Work_Experience  155 non-null    object
dtypes: object(3)
memory usage: 3.8+ KB


In [ ]:
df.describe()

,Level,Job_Title,Work_Experience
count,155,155,155
unique,3,3,155
top,Senior,DevOps Engineer,Data Engineering Intern \nJuly 2022 – July 202...
freq,65,61,1


In [ ]:
df['Level'].value_counts()


Senior         65
Entry-Level    48
Junior         42
Name: Level, dtype: int64

In [ ]:
df['Job_Title'].value_counts()

DevOps Engineer              61
Machine Learning Engineer    50
Data Engineer                44
Name: Job_Title, dtype: int64

We can see we have a more or less balanced dataset.

In [ ]:
# Text Preprocessing Function
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation and special characters
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenization
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]
    # Rejoin tokens into a preprocessed string
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
df['Preprocessed_Resume'] = df['Work_Experience'].apply(preprocess_text)


In [ ]:
df

,Level,Job_Title,Work_Experience,Preprocessed_Resume
0,Entry-Level,Data Engineer,Data Engineering Intern \nJuly 2022 – July 202...,data engin intern juli 2022 juli 2023 1 year a...
1,Entry-Level,Data Engineer,5/2022 to 5/2023 1 year\nData Warehouse Engine...,52022 52023 1 year data warehous engin tucson ...
2,Entry-Level,Data Engineer,"Data Engineer\nApr 2022 Present , 1 yr 4 mos\n...",data engin apr 2022 present 1 yr 4 mo unit sta...
3,Entry-Level,Data Engineer,"Data Engineer\nJul 2018 Jun 2019 , 1 yr\nSan F...",data engin jul 2018 jun 2019 1 yr san francisc...
4,Entry-Level,Data Engineer,"Data Engineer\nOct 2017 Jun 2019 , 1 yrs 9 mos...",data engin oct 2017 jun 2019 1 yr 9 mo bangalo...
...,...,...,...,...
150,Senior,DevOps Engineer,DevOps Engineer\nJan 2016 to Present (7 years ...,devop engin jan 2016 present 7 year 7 month cu...
151,Senior,Machine Learning Engineer,"Design, prototype, evaluate and deploy machine...",design prototyp evalu deploy machin learn mode...
152,Entry-Level,Data Engineer,"Data Engineer\nOct 2018 Jun 2019, 1 yr 9 mos\n...",data engin oct 2018 jun 2019 1 yr 9 mo unit st...
153,Entry-Level,Machine Learning Engineer,"Jul 2021 Nov 2022, 1 yr 5 mos\nDearborn, Michi...",jul 2021 nov 2022 1 yr 5 mo dearborn michigan ...


In [ ]:
# Data Preprocessing
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['Preprocessed_Resume'])

In [ ]:
X

<155x2312 sparse matrix of type '<class 'numpy.float64'>'
	with 12760 stored elements in Compressed Sparse Row format>

In [ ]:
y = df['Job_Title']


In [ ]:
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Model 1 - Support Vector Classifier

SVC are supervised ML algorithms used for classification. They are based on the concept of support vectors. SVC tries to find the optimal hyperplane that maximizes the margin between classes, making an effective option for binary and multiclass classification.

SVCs are efficient in high-dimensional spaces making them well suited for text classification.



In [ ]:
# Model Selection and Training
model_1 = SVC()
model_1.fit(X_train, y_train)

SVC()

In [ ]:
# Model Evaluation
y_pred = model_1.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
                           precision    recall  f1-score   support

            Data Engineer       1.00      0.80      0.89        10
          DevOps Engineer       0.87      0.93      0.90        14
Machine Learning Engineer       0.88      1.00      0.93         7

                 accuracy                           0.90        31
                macro avg       0.91      0.91      0.91        31
             weighted avg       0.91      0.90      0.90        31



We will now perform hyperparameter tuning and compare the results.

In [ ]:
# Hyperparameter Tuning using Grid Search
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': [0.1, 1, 'scale']
}

In [ ]:
model_1_tuned = SVC()
grid_search = GridSearchCV(estimator=model_1_tuned, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10], 'gamma': [0.1, 1, 'scale'],
                         'kernel': ['linear', 'rbf']})

In [ ]:
# Best hyperparameters from Grid Search
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)


Best Hyperparameters: {'C': 10, 'gamma': 0.1, 'kernel': 'linear'}


In [ ]:
# Model Evaluation with Best Hyperparameters
best_model = SVC(**best_params)
best_model.fit(X_train, y_train)

SVC(C=10, gamma=0.1, kernel='linear')

In [ ]:
y_pred = best_model.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
                           precision    recall  f1-score   support

            Data Engineer       1.00      0.80      0.89        10
          DevOps Engineer       0.87      0.93      0.90        14
Machine Learning Engineer       0.88      1.00      0.93         7

                 accuracy                           0.90        31
                macro avg       0.91      0.91      0.91        31
             weighted avg       0.91      0.90      0.90        31



# Model 2 - Multinomial Naive Bayes classifier



In [ ]:
model_2 = MultinomialNB()
model_2.fit(X_train, y_train)



MultinomialNB()

In [ ]:
# Model Evaluation
y_pred = model_2.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
                           precision    recall  f1-score   support

            Data Engineer       1.00      0.80      0.89        10
          DevOps Engineer       0.88      1.00      0.93        14
Machine Learning Engineer       1.00      1.00      1.00         7

                 accuracy                           0.94        31
                macro avg       0.96      0.93      0.94        31
             weighted avg       0.94      0.94      0.93        31



In [ ]:
# Hyperparameter Tuning using Grid Search with Cross-Validation
param_grid = {'alpha': [0.1, 1, 10]}

model_2_tuned = MultinomialNB()
grid_search = GridSearchCV(estimator=model_2_tuned, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Best hyperparameters from Grid Search
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)


Best Hyperparameters: {'alpha': 1}


In [ ]:
# Model Evaluation with Best Hyperparameters
best_model_2 = MultinomialNB(alpha=best_params['alpha'])
best_model_2.fit(X_train, y_train)

y_pred = best_model_2.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
                           precision    recall  f1-score   support

            Data Engineer       1.00      0.80      0.89        10
          DevOps Engineer       0.88      1.00      0.93        14
Machine Learning Engineer       1.00      1.00      1.00         7

                 accuracy                           0.94        31
                macro avg       0.96      0.93      0.94        31
             weighted avg       0.94      0.94      0.93        31

